## 决策树的构造
### 信息增益

In [1]:
from math import log
import operator

In [2]:
def calcShannonEnt(dataSet):
    # 数据量
    numEntries = len(dataSet)
    # 字典，用于存放每个类别出现的次数
    labelCounts = {}
    for featVec in dataSet:
        currentLabel = featVec[-1]  # 最后的值为label
        # 如果当前的label不在字典中，将其添加到字典中
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        
        # 计数
        labelCounts[currentLabel] += 1
    
    shannonEnt = 0.0
    for key in labelCounts:
        # 出现频次
        prob = float(labelCounts[key]) / numEntries
        # 熵
        shannonEnt -= prob * log(prob, 2)
    return shannonEnt

In [3]:
def createDataSet():
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    labels = ['no surfacing', 'flippers']
    return dataSet, labels

In [4]:
myDat, labels = createDataSet()

In [5]:
myDat

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [6]:
labels

['no surfacing', 'flippers']

In [7]:
calcShannonEnt(myDat)

0.9709505944546686

In [8]:
# 新增一个类别，重新计算熵
myDat[0][-1] = 'maybe'
myDat

[[1, 1, 'maybe'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [9]:
calcShannonEnt(myDat)

1.3709505944546687

### 划分数据集

In [10]:
def splitDataSet(dataSet, axis, value):
    """
    dataSet: 数据集
    axis: 特征列
    value: 特征的取值
    """
    retDataSet = []
    for featVec in dataSet:
        # 对该特征取值为value的行添加到retDataSet中，同时去除该取值
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

In [11]:
myDat, labels = createDataSet()

In [12]:
myDat

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [13]:
# 第一个特征，取值为1
splitDataSet(myDat, 0, 1)

[[1, 'yes'], [1, 'yes'], [0, 'no']]

In [14]:
# 第一个特征，取值为0
splitDataSet(myDat, 0, 0)

[[1, 'no'], [1, 'no']]

In [15]:
def chooseBestFeatureToSplit(dataSet):
    # 特征个数
    numFeatures = len(dataSet[0]) - 1
    # 经验熵
    baseEntropy = calcShannonEnt(dataSet)
    
    bestInfoGain = 0.0
    bestFeature = -1
    for i in range(numFeatures):
        # 以下两行找到特征的所有取值
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)
        
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            # 频率
            prob = len(subDataSet) / float(len(dataSet))
            # 经验条件熵
            newEntropy += prob * calcShannonEnt(subDataSet)
        # 信息增益=经验熵-经验条件熵
        infoGain = baseEntropy - newEntropy
        if (infoGain > bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature

In [16]:
myDat, labels = createDataSet()
myDat

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [17]:
chooseBestFeatureToSplit(myDat)

0

第0个特征是最好的划分数据集的特征

### 递归构建决策树

In [18]:
def majorityCnt(classList):
    classCount = {}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.iteritems(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

In [19]:
def createTree(dataSet,labels):
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList): 
        return classList[0]
    if len(dataSet[0]) == 1:
        return majorityCnt(classList)
    bestFeat = chooseBestFeatureToSplit(dataSet)
    print('bestFeat')
    print(bestFeat)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value),subLabels)
    return myTree   

myDat, labels = createDataSet()
myTree = createTree(myDat, labels)
myTree

bestFeat
0
bestFeat
0


{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}